In [6]:
import numpy as np
data=np.load('data_more.npy')
target=np.load('target_more.npy')

In [7]:
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D,Activation,MaxPooling2D
from keras.utils import normalize
from keras.layers import Concatenate
from keras import Input
from keras.callbacks import ModelCheckpoint


In [42]:

input_shape=data.shape[1:] #50,50,1
inp=Input(shape=input_shape)
convs=[]
#parrale 2d conv layer 128@3x3 128@5x5 128@7x7
parralel_kernels=[3,5,7]
for k in range(len(parralel_kernels)):

    conv = Conv2D(128,parralel_kernels[k],border_mode='same',activation='relu',input_shape=input_shape,strides=1)(inp)
    convs.append(conv)

out = Concatenate()(convs)
conv_model = Model(input=inp, output=out)

model = Sequential()
model.add(conv_model)

model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
#use dropout for minimize overfitting
model.add(Dropout(0.5))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))

#use softmax as activiton of output layer since probability we need COVID19 & Negative
model.add(Dense(2,input_dim=128,activation='softmax'))

#categorical crossentropy use for specifictaion type problem
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model.summary()

<ipython-input-42-f471ceed3dfd>:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, 3, activation="relu", input_shape=(100, 100,..., strides=1, padding="same")`
  conv = Conv2D(128,parralel_kernels[k],border_mode='same',activation='relu',input_shape=input_shape,strides=1)(inp)
<ipython-input-42-f471ceed3dfd>:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, 5, activation="relu", input_shape=(100, 100,..., strides=1, padding="same")`
  conv = Conv2D(128,parralel_kernels[k],border_mode='same',activation='relu',input_shape=input_shape,strides=1)(inp)
<ipython-input-42-f471ceed3dfd>:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, 7, activation="relu", input_shape=(100, 100,..., strides=1, padding="same")`
  conv = Conv2D(128,parralel_kernels[k],border_mode='same',activation='relu',input_shape=input_shape,strides=1)(inp)
<ipython-input-42-f471ceed3dfd>:12: UserWarning: Update your `Model` call to the Keras 2 API: `Mo

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_3 (Model)              (None, 100, 100, 384)     11008     
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 98, 98, 64)        221248    
_________________________________________________________________
activation_5 (Activation)    (None, 98, 98, 64)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 49, 49, 64)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 47, 47, 32)        18464     
_________________________________________________________________
activation_6 (Activation)    (None, 47, 47, 32)        0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 23, 23, 32)       

In [43]:
from sklearn.model_selection import train_test_split
#as we see 10% testing go to testing and 90% go to the training 
train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.1)

In [44]:
#save all the nueral model here then moniter validation and then we save best only
#val-loss for minimum validation loss
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
#10 percent use as validation
#increasing epochs we increase the neural neural
history=model.fit(train_data,train_target,epochs=20,callbacks=[checkpoint],validation_split=0.1)

Train on 707 samples, validate on 79 samples
Epoch 1/20
707/707 [==============================] - 295s 418ms/step - loss: 0.6495 - accuracy: 0.6040 - val_loss: 0.2532 - val_accuracy: 0.9367
Epoch 2/20
707/707 [==============================] - 230s 325ms/step - loss: 0.2394 - accuracy: 0.9137 - val_loss: 0.1167 - val_accuracy: 0.9620
Epoch 3/20
707/707 [==============================] - 203s 287ms/step - loss: 0.2201 - accuracy: 0.9364 - val_loss: 0.1001 - val_accuracy: 0.9494
Epoch 4/20
707/707 [==============================] - 224s 317ms/step - loss: 0.1879 - accuracy: 0.9505 - val_loss: 0.0750 - val_accuracy: 0.9620
Epoch 5/20
707/707 [==============================] - 220s 312ms/step - loss: 0.1363 - accuracy: 0.9604 - val_loss: 0.1170 - val_accuracy: 0.9747
Epoch 6/20
707/707 [==============================] - 191s 270ms/step - loss: 0.0647 - accuracy: 0.9745 - val_loss: 0.0608 - val_accuracy: 0.9620
Epoch 7/20
707/707 [==============================] - 186s 262ms/step - loss: 0

In [45]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('model_keras1.tflite', 'wb') as f:
  f.write(tflite_model)

AttributeError: 'Sequential' object has no attribute '_get_save_spec'

In [ ]:
plt.plot(history.history['loss'],'r',label='training loss')
plt.plot(history.history['val_loss'],label='validation loss')
plt.xlabel('# epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

plt.plot(history.history['accuracy'],'r',label='training accuracy')
plt.plot(history.history['val_accuracy'],label='validation accuracy')
plt.xlabel('# epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [40]:
print(model.evaluate(test_data,test_target))

InvalidArgumentError:  Incompatible shapes: [32,2] vs. [32,100,100,1]
	 [[node mean_squared_error/SquaredDifference (defined at <ipython-input-40-a15c08f78d9b>:1) ]] [Op:__inference_test_function_5971]

Function call stack:
test_function
